## Import


In [1]:
import numpy as np
import pandas as pd

## Daten einlesen


In [2]:
col_names = ["contour number", "aspect ratio", "extent", "Blue", "Green", "Red", "Hue"]
data = pd.read_csv("output.csv", index_col=0)
# besser als excel weil csv weniger Metadaten speichert und so bei vielen Bildern Rechenzeit reduziert wird
# data = pd.read_excel("output.xlsx", skiprows=1, header=None, names=col_names)
data.head(30)

,contour number,aspect ratio,extent,Blue,Green,Red,Hue,class
0,28,0.923077,0.803775,42.769815,97.239460,115.102867,19.023609,0
1,68,0.979804,0.836557,63.621205,104.295904,104.446747,29.446747,0
2,51,0.760961,0.823015,59.701164,99.514131,103.025769,32.987531,0
3,31,0.957265,0.767948,50.077372,116.466180,137.658394,22.445255,0
4,50,0.828162,0.845766,60.658955,104.919383,113.961094,34.713985,0
5,47,0.890923,0.828608,52.141768,97.165096,107.391655,28.818304,0
6,45,0.950329,0.748096,48.570281,121.473896,109.674699,15.439759,0
7,39,0.901044,0.802361,56.255943,103.945878,107.513404,25.372281,0
8,53,0.539305,0.713518,44.690566,132.909434,108.475472,15.988679,0
9,47,0.915618,0.802961,51.757838,96.647207,110.642883,25.927928,0


## Klasse für Knoten


In [3]:
class Node:
    def __init__(
        self,
        feature_index=None,
        threshold=None,
        left=None,
        right=None,
        info_gain=None,
        value=None,
    ):
        """constructor"""

        # für decision node
        self.feature_index = feature_index
        self.threshold = threshold
        self.left = left
        self.right = right
        self.info_gain = info_gain

        # für leaf node
        self.value = value

## Klasse für Erstellung des Baumes


In [4]:
class DecisionTreeClassifier:
    def __init__(self, min_samples_split=2, max_depth=2):
        """constructor"""

        # Initialisiere root des decision tree, wird benötigt um tree zu durchlaufen
        self.root = None

        # node wird nicht weiter verästelt wenn:
        self.min_samples_split = min_samples_split  # wenn Anzahl der Samples aus den trainingsdaten im node weniger als dieser wert wird, wird node zu leaf node
        self.max_depth = max_depth  # wenn max anzahl an verketteten Ebenen erreicht ist, wird node zu leaf node

    def build_tree(self, dataset, curr_depth=0):
        """rekursive Funktion um decision tree aufzubauen"""
        # Array an Trainingsdaten wird aufgespalten: Merkmale X werden von Klassenzuordnung Y (letzte Spalte) getrennt
        X, Y = dataset[:, :-1], dataset[:, -1]
        # Abfrage: num_samples= Anzahl Zeilen der features; num_features= Anzahl Spalten der features
        num_samples, num_features = np.shape(X)

        # weiter verästeln, bis Stop-Bedingung
        if num_samples >= self.min_samples_split and curr_depth <= self.max_depth:
            # finde beste Aufteilung (maximiere information gain)
            best_split = self.get_best_split(dataset, num_samples, num_features)
            # überprüfe ob information gain positiv, wenn nicht besteht Knoten nur noch aus einer Klasse
            if best_split["info_gain"] > 0:
                # Rekursion für linke Seite des Baumes bis überall leaf nodes
                left_subtree = self.build_tree(
                    best_split["dataset_left"], curr_depth + 1
                )
                # Rekursion für rechte Seite
                right_subtree = self.build_tree(
                    best_split["dataset_right"], curr_depth + 1
                )
                # gebe decision node zurück
                return Node(
                    best_split["feature_index"],
                    best_split["threshold"],
                    left_subtree,
                    right_subtree,
                    best_split["info_gain"],
                )

        # berechne leaf node
        leaf_value = self.calculate_leaf_value(Y)
        # gebe leaf node zurück
        return Node(value=leaf_value)

    def get_best_split(self, dataset, num_samples, num_features):
        """function to find the best split"""

        # dictionary um beste Teilung zu speichern
        best_split = {}
        max_info_gain = -float("inf")

        # Durch alle Merkmale iterieren
        for feature_index in range(num_features):
            feature_values = dataset[:, feature_index]
            possible_thresholds = np.unique(feature_values)
            # iterieren durch alle Merkmalswerte im Datensatz
            for threshold in possible_thresholds:
                # Teilung berechnen
                dataset_left, dataset_right = self.split(
                    dataset, feature_index, threshold
                )
                # resultierende Datasets dürfen nicht 0 sein
                if len(dataset_left) > 0 and len(dataset_right) > 0:
                    # Extrahieren der Klassen aus dem übrig gegbliebenen dataset
                    y, left_y, right_y = (
                        dataset[:, -1],
                        dataset_left[:, -1],
                        dataset_right[:, -1],
                    )
                    # Berechnung information gain
                    curr_info_gain = self.information_gain(y, left_y, right_y, "gini")
                    # Aktualisieren der besten Teilung wenn gain größer ist als aktuelles maximun
                    if curr_info_gain > max_info_gain:
                        best_split["feature_index"] = feature_index
                        best_split["threshold"] = threshold
                        best_split["dataset_left"] = dataset_left
                        best_split["dataset_right"] = dataset_right
                        best_split["info_gain"] = curr_info_gain
                        max_info_gain = curr_info_gain

        # Beste Teilung zurück geben
        return best_split

    def split(self, dataset, feature_index, threshold):
        """function to split the data"""

        dataset_left = np.array(
            [row for row in dataset if row[feature_index] <= threshold]
        )
        dataset_right = np.array(
            [row for row in dataset if row[feature_index] > threshold]
        )
        return dataset_left, dataset_right

    def information_gain(self, parent, l_child, r_child, mode="entropy"):
        """function to compute information gain"""

        weight_l = len(l_child) / len(parent)
        weight_r = len(r_child) / len(parent)
        if mode == "gini":
            gain = self.gini_index(parent) - (
                weight_l * self.gini_index(l_child)
                + weight_r * self.gini_index(r_child)
            )
        else:
            gain = self.entropy(parent) - (
                weight_l * self.entropy(l_child) + weight_r * self.entropy(r_child)
            )
        return gain

    def entropy(self, y):
        """function to compute entropy"""

        class_labels = np.unique(y)
        entropy = 0
        for cls in class_labels:
            p_cls = len(y[y == cls]) / len(y)
            entropy += -p_cls * np.log2(p_cls)
        return entropy

    def gini_index(self, y):
        """function to compute gini index"""

        class_labels = np.unique(y)
        gini = 0
        for cls in class_labels:
            p_cls = len(y[y == cls]) / len(y)
            gini += p_cls**2
        return 1 - gini

    def calculate_leaf_value(self, Y):
        """function to compute leaf node"""

        Y = list(Y)
        return max(Y, key=Y.count)

    def print_tree(self, tree=None, indent=" "):
        """function to print the tree"""

        if not tree:
            tree = self.root

        if tree.value is not None:
            print(tree.value)

        else:
            print(
                "X_" + str(tree.feature_index),
                "<=",
                tree.threshold,
                "?",
                tree.info_gain,
            )
            print("%sleft:" % (indent), end="")
            self.print_tree(tree.left, indent + indent)
            print("%sright:" % (indent), end="")
            self.print_tree(tree.right, indent + indent)

    def fit(self, X, Y):
        """function to train the tree"""

        dataset = np.concatenate((X, Y), axis=1)
        self.root = self.build_tree(dataset)

    def predict(self, X):
        """function to predict new dataset"""

        predictions = [self.make_prediction(x, self.root) for x in X]
        return predictions

    def make_prediction(self, x, tree):
        """function to predict a single data point"""

        if tree.value != None:
            return tree.value
        feature_val = x[tree.feature_index]
        if feature_val <= tree.threshold:
            return self.make_prediction(x, tree.left)
        else:
            return self.make_prediction(x, tree.right)

## Aufteilung der Daten in Trainings- und Testdaten


In [5]:
X = data.iloc[
    :, :-1
].values  # erstes ":" für alle Zeilen, nach dem Komma ":-1" = "alle Spalten bis auf die letzte", Bildung np-Array durch ".values"
Y = data.iloc[:, -1].values.reshape(
    -1, 1
)  # nur letzte Spalte. "reshape" konvertiert Vektor in Spaltenvektor mit 1 Spalte. durch "-1" werden Zeilen automatisch berechnet
from sklearn.model_selection import train_test_split


# Aufteilen der Daten in Trainings-und Testdaten

X_train, X_test, Y_train, Y_test = train_test_split(

    X, Y, test_size=0.2, random_state=41

)

## Fit the model


In [6]:
classifier = DecisionTreeClassifier(min_samples_split=3, max_depth=3)
classifier.fit(X_train, Y_train)
classifier.print_tree()

X_1 <= 0.3765165413130649 ? 0.2876945336622756
 left:X_0 <= 24.0 ? 0.06243496357960443
  left:0.0
  right:1.0
 right:X_5 <= 101.70281124497993 ? 0.34060712503336904
  left:X_2 <= 0.5728302062435169 ? 0.05709342560553643
    left:0.0
    right:2.0
  right:X_0 <= 58.0 ? 0.07644464783458854
    left:X_3 <= 39.443708609271525 ? 0.0448458424166923
        left:2.0
        right:0.0
    right:X_1 <= 0.8673017867965745 ? 0.375
        left:2.0
        right:0.0


## Test the model


In [7]:
Y_pred = classifier.predict(X_test)
from sklearn.metrics import accuracy_score

accuracy_score(Y_test, Y_pred)

0.8928571428571429